In [1]:
import numpy as np
import pandas as pd

from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score, confusion_matrix
from sklearn.preprocessing import label_binarize

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

from art.attacks.evasion import SimBA, SpatialTransformation, DeepFool, BasicIterativeMethod, FastGradientMethod, ProjectedGradientDescent
from art.estimators.classification import PyTorchClassifier

from art.defences.trainer import AdversarialTrainerAWPPyTorch

import time

/opt/conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
head = {
            "model" : '',
            "attack_model": '',
            'epsilon': '',
            'Accuracy': '',
            'Macro Precision': '',
            'Weighted Precision': '',
            'Macro Recall': '',
            'Weighted Recall': '',
            'Macro F1': '',
            'Weighted F1': '',
            # 'Macro AUC': '',
            # 'Weighted AUC': '',
            # 'TPR': '',
            # 'FNR': '',
            # 'TNR': '',
            # 'FPR': '',
        }
head = pd.DataFrame([head])
head.to_csv("/home/jovyan/Defense/AWP/AWP_AutoPGD.csv", mode='a', index=False)

In [3]:
from sklearn.metrics import confusion_matrix, precision_score, accuracy_score, f1_score,recall_score, roc_auc_score
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

def calculate_performance_metrics(x_test_adv, y_test, classifier, model_name, attack_name, epsilon):

    start_time = time.time()

    preds = np.argmax(classifier.predict(x_test_adv), axis=1)
    # preds = classifier.predict(x_test_adv)

    y_hat = preds
    # y_test_arg = np.argmax(y_test, axis=-1)
    y_test_arg = y_test


    accuracy = accuracy_score(y_test_arg, y_hat)
    precision_macro = precision_score(y_test_arg, y_hat, average='macro')
    # precision_micro = precision_score(y_test_arg, y_hat, average='micro')
    precision_weighted = precision_score(y_test_arg, y_hat, average='weighted')
    f1_macro = f1_score(y_test_arg, y_hat, average='macro')
    # f1_micro = f1_score(y_test_arg, y_hat, average='micro')
    f1_weighted = f1_score(y_test_arg, y_hat, average='weighted')
    recall_macro = recall_score(y_test_arg, y_hat, average='macro')
    # recall_micro = recall_score(y_test_arg, y_hat, average='micro')
    recall_weighted = recall_score(y_test_arg, y_hat, average='weighted')


    print(f"epsilon = {epsilon}")

    print(f"Accuracy: {accuracy}")
    # print(f"Precision(micro,macro,weighted): {precision_micro},{precision_macro},{precision_weighted}")
    # print(f"F1(micro,macro,weighted: {f1_micro},{f1_macro},{f1_weighted}")
    # print(f"Recall(micro,macro,weighted: {recall_micro},{recall_macro},{recall_weighted}")
    print(f"Precision(macro,weighted): {precision_macro},{precision_weighted}")
    print(f"F1(macro,weighted: {f1_macro},{f1_weighted}")
    print(f"Recall(macro,weighted: {recall_macro},{recall_weighted}")


    
    metrics = {
        "model" : model_name,
        "attack_model" : attack_name,
        "epsilon" : epsilon,
        "Accuracy": accuracy,
        # "Precision-micro": precision_micro,
        "Precision-macro": precision_macro,
        "Precision-weighted": precision_weighted,
        # "Recall_micro": recall_micro,
        "Recall_macro": recall_macro,
        "Recall_weighted": recall_weighted,
        
        # "F1-micro": f1_micro,
        "F1-macro": f1_macro,
        "F1-weighted": f1_weighted,

    }
    metrics_df = pd.DataFrame([metrics])
    metrics_df.to_csv("/home/jovyan/Defense/AWP/AWP_AutoPGD.csv", mode='a', index=False, header=False)

    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Time: {elapsed_time:.2f} second\n\n")

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

Using cuda device


In [5]:
x_test = np.load('/home/jovyan/Wustl_iiot/x_test.npy')
x_train = np.load('/home/jovyan/Wustl_iiot/x_train.npy')
x_val = np.load('/home/jovyan/Wustl_iiot/x_val.npy')
y_test = np.load('/home/jovyan/Wustl_iiot/y_test.npy')
y_train = np.load('/home/jovyan/Wustl_iiot/y_train.npy')
y_val = np.load('/home/jovyan/Wustl_iiot/y_val.npy')


# train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
# train_loader = DataLoader(train_dataset, batch_size=100, shuffle=True)

# val_dataset = TensorDataset(x_val_tensor, y_val_tensor)
# val_loader = DataLoader(val_dataset, batch_size=100, shuffle=True)

In [6]:
input_shape = x_train.shape[1]
output_shape = len(np.unique(y_train))

In [7]:
class DNNModel(nn.Module):
    def __init__(self, input_size, output_size):
        super(DNNModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, 30)
        self.fc3 = nn.Linear(30, 20)
        self.fc4 = nn.Linear(20, output_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

In [8]:
model = DNNModel(input_size=input_shape, output_size=output_shape).to(device)
model.load_state_dict(torch.load("/home/jovyan/Wustl_iiot/transfer_attack/dnn_pytorch.pt"))

<All keys matched successfully>

In [9]:
x_train = x_train.astype(np.float32)
x_test = x_test.astype(np.float32)
x_val = x_val.astype(np.float32)

In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
    
classifier = PyTorchClassifier(
    model=model,
    clip_values=(-5, 5),
    loss=criterion,
    optimizer=optimizer,
    input_shape=(input_shape,),
    nb_classes=output_shape,
    device_type='gpu'
)

In [14]:
from art.attacks.evasion import SimBA, SpatialTransformation, DeepFool, BasicIterativeMethod, FastGradientMethod, ProjectedGradientDescent, AutoProjectedGradientDescent

# FGSM = FastGradientMethod(estimator=classifier, eps=0.3)
# BIM = BasicIterativeMethod(estimator=classifier, eps=0.3, max_iter=3)
# PGD = ProjectedGradientDescent(estimator=classifier, eps=0.3, max_iter=3)
# DF = DeepFool(classifier, epsilon=0.3, verbose=False, max_iter=3)
AutoPGD = AutoProjectedGradientDescent(classifier, eps=0.3, max_iter = 10, batch_size=128, verbose = False)


In [15]:
# FGSM_Trainer = AdversarialTrainerAWPPyTorch(classifier, classifier, FGSM, "TRADES", 0.1, 6.0, 0)
# BIM_Trainer = AdversarialTrainerAWPPyTorch(classifier, classifier, BIM, "TRADES", 0.1, 6.0, 0)
# PGD_Trainer = AdversarialTrainerAWPPyTorch(classifier, classifier, PGD, "TRADES", 0.1, 6.0, 0)
# DF_Trainer = AdversarialTrainerAWPPyTorch(classifier, classifier, DF, "TRADES", 0.1, 6.0, 0)
AutoPGD_Trainer = AdversarialTrainerAWPPyTorch(classifier, classifier, AutoPGD, "TRADES", 0.1, 6.0, 0)

In [14]:
# start_time = time.time()
# FGSM_Trainer.fit(x_train, y_train, epochs=10, batch_size=128,
#                      validation_data=(x_val, y_val))
# end_time = time.time()
# elapsed_time = end_time - start_time
# print(f"Time: {elapsed_time:.2f} second\n\n")

In [15]:
# start_time = time.time()
# BIM_Trainer.fit(x_train, y_train, epochs=10, batch_size=128,
#                      validation_data=(x_val, y_val))
# end_time = time.time()
# elapsed_time = end_time - start_time
# print(f"Time: {elapsed_time:.2f} second\n\n")

In [16]:
# start_time = time.time()
# PGD_Trainer.fit(x_train, y_train, epochs=10, batch_size=128,
#                      validation_data=(x_val, y_val))
# end_time = time.time()
# elapsed_time = end_time - start_time
# print(f"Time: {elapsed_time:.2f} second\n\n")

In [ ]:
# start_time = time.time()
# DF_Trainer.fit(x_train, y_train, epochs=5, batch_size=128,
#                      validation_data=(x_val, y_val))
# end_time = time.time()
# elapsed_time = end_time - start_time
# print(f"Time: {elapsed_time:.2f} second\n\n")

In [16]:
start_time = time.time()
AutoPGD_Trainer.fit(x_train, y_train, epochs=5, batch_size=128,
                     validation_data=(x_val, y_val))
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time: {elapsed_time:.2f} second\n\n")

Adversarial Training AWP with TRADES - Epochs: 100%|██████████| 20/20 [5:27:57<00:00, 983.88s/it]   

Time: 19677.53 second




In [16]:
# calculate_performance_metrics(x_test, y_test, BIM_Trainer, 'BIM_Trainer', 'baseline', '0')
# calculate_performance_metrics(x_test, y_test, FGSM_Trainer, 'FGSM_Trainer', 'baseline', '0')
# calculate_performance_metrics(x_test, y_test, PGD_Trainer, 'PGD_Trainer', 'baseline', '0')

epsilon = 0
Accuracy: 0.9982953093783115
Precision(macro,weighted): 0.5947964623438752,0.9979435892168569
F1(macro,weighted: 0.5763951699364166,0.9980405778895473
Recall(macro,weighted: 0.5620145631067961,0.9982953093783115
Time: 2.71 second




/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# epsilon_values = [0.01, 0.1, 0.2, 0.3]

# Iterate over epsilon values
# for epsilon in epsilon_values:
#     filename = f'/home/jovyan/Wustl_iiot/transfer_attack/x_test_adv_BIM_eps_{epsilon}.npy'
#     x_test_adv = np.load(filename)

#     calculate_performance_metrics(x_test_adv, y_test, BIM_Trainer, 'BIM_Trainer', 'BIM', epsilon)

# for epsilon in epsilon_values:
#     filename = f'/home/jovyan/Wustl_iiot/transfer_attack/x_test_adv_FGSM_eps_{epsilon}.npy'
#     x_test_adv = np.load(filename)

#     calculate_performance_metrics(x_test_adv, y_test, FGSM_Trainer, 'FGSM_Trainer', 'FGSM', epsilon)

# for epsilon in epsilon_values:
#     filename = f'/home/jovyan/Wustl_iiot/transfer_attack/x_test_adv_PGD_eps_{epsilon}.npy'
#     x_test_adv = np.load(filename)

#     calculate_performance_metrics(x_test_adv, y_test, PGD_Trainer, 'PGD_Trainer', 'PGD', epsilon)

In [17]:
torch.save(AutoPGD_Trainer, "/home/jovyan/Defense/AWP/AWP_AutoPGD.pt")

In [ ]:
# skip start here

In [18]:
AutoPGD_Trainer = torch.load("/home/jovyan/Defense/AWP/AWP_AutoPGD.pt")


In [19]:
calculate_performance_metrics(x_test, y_test, AutoPGD_Trainer, 'AutoPGD_Trainer', 'baseline', '0')

epsilon_values = [0.01, 0.1, 0.2, 0.3]

# Iterate over epsilon values
print("start BIM")
for epsilon in epsilon_values:
    filename = f'/home/jovyan/Wustl_iiot/transfer_attack/x_test_adv_BIM_eps_{epsilon}.npy'
    x_test_adv = np.load(filename)

    calculate_performance_metrics(x_test_adv, y_test, AutoPGD_Trainer, 'AutoPGD_Trainer', 'BIM', epsilon)

print("start FGSM")
for epsilon in epsilon_values:
    filename = f'/home/jovyan/Wustl_iiot/transfer_attack/x_test_adv_FGSM_eps_{epsilon}.npy'
    x_test_adv = np.load(filename)

    calculate_performance_metrics(x_test_adv, y_test, AutoPGD_Trainer, 'AutoPGD_Trainer', 'FGSM', epsilon)

print("start PGD")
for epsilon in epsilon_values:
    filename = f'/home/jovyan/Wustl_iiot/transfer_attack/x_test_adv_PGD_eps_{epsilon}.npy'
    x_test_adv = np.load(filename)

    calculate_performance_metrics(x_test_adv, y_test, AutoPGD_Trainer, 'AutoPGD_Trainer', 'PGD', epsilon)

epsilon_values = [0.01, 0.1, 0.2, 0.3]

print("start DF")
for epsilon in epsilon_values:
    filename = f'/home/jovyan/Wustl_iiot/transfer_attack/x_test_adv_DF_eps_{epsilon}.npy'
    x_test_adv = np.load(filename)

    calculate_performance_metrics(x_test_adv, y_test, AutoPGD_Trainer, 'AutoPGD_Trainer', 'DF', epsilon)
    
# Iterate over epsilon values
print("start AutoPGD")
for epsilon in epsilon_values:
    filename = f'/home/jovyan/Wustl_iiot/transfer_attack/x_test_adv_AutoPGD_eps_{epsilon}.npy'
    x_test_adv = np.load(filename)

    calculate_performance_metrics(x_test_adv, y_test, AutoPGD_Trainer, 'AutoPGD_Trainer', 'AutoPGD', epsilon)

print("start BA")
for epsilon in epsilon_values:
    filename = f'/home/jovyan/Wustl_iiot/transfer_attack/x_test_adv_BA_eps_{epsilon}.npy'
    x_test_adv = np.load(filename)

    calculate_performance_metrics(x_test_adv, y_test, AutoPGD_Trainer, 'AutoPGD_Trainer', 'BA', epsilon)

print("start HopSkipJump")
filename = f'/home/jovyan/Wustl_iiot/transfer_attack/x_test_adv_HopSkipJump_eps_0.npy'
x_test_adv = np.load(filename)

calculate_performance_metrics(x_test_adv, y_test, AutoPGD_Trainer, 'AutoPGD_Trainer', 'HopSkipJump', '0')

print("start ZOO")
ZOO_epsilon_values = [0, 0.01, 0.1, 0.2, 0.3]
# Iterate over epsilon values
for epsilon in ZOO_epsilon_values:
    filename = f'/home/jovyan/Wustl_iiot/transfer_attack/x_test_adv_ZOO_eps_{epsilon}.npy'
    x_test_adv = np.load(filename)

    calculate_performance_metrics(x_test_adv, y_test, AutoPGD_Trainer, 'AutoPGD_Trainer', 'ZOO', epsilon)

/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


epsilon = 0
Accuracy: 0.9992963439202858
Precision(macro,weighted): 0.7945788214251042,0.9991192183142162
F1(macro,weighted: 0.7487562825516576,0.9991946828363167
Recall(macro,weighted: 0.7176624346527258,0.9992963439202858
Time: 2.76 second


start BIM


/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


epsilon = 0.01
Accuracy: 0.9988607472995104
Precision(macro,weighted): 0.792965392245776,0.9986855572595491
F1(macro,weighted: 0.6910966996067209,0.9987192954492665
Recall(macro,weighted: 0.6492896117635131,0.9988607472995104
Time: 2.74 second




/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


epsilon = 0.1
Accuracy: 0.9989696464547042
Precision(macro,weighted): 0.7924756192725059,0.9987894226788265
F1(macro,weighted: 0.7426650276781599,0.9988544798792183
Recall(macro,weighted: 0.7081964152352501,0.9989696464547042
Time: 2.73 second




/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


epsilon = 0.2
Accuracy: 0.9989612695966124
Precision(macro,weighted): 0.7924441416550259,0.9987810342976272
F1(macro,weighted: 0.7425162248841931,0.998845645430917
Recall(macro,weighted: 0.7079536967886482,0.9989612695966124
Time: 2.75 second


epsilon = 0.3
Accuracy: 0.9989612695966124
Precision(macro,weighted): 0.7924441416550259,0.9987810342976272
F1(macro,weighted: 0.7425162248841931,0.998845645430917
Recall(macro,weighted: 0.7079536967886482,0.9989612695966124
Time: 2.74 second




/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
